In [1]:
from transformers import BertTokenizer, BertForTokenClassification, BertTokenizerFast
from transformers import Trainer, TrainingArguments, DataCollatorForTokenClassification, pipeline
from sklearn.model_selection import train_test_split
import torch
import json
import evaluate
import numpy
from random import shuffle

/root/miniconda3/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/root/miniconda3/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 加载BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('NER_model')

In [3]:

# 加载BERT模型，用于NER任务微调
model = BertForTokenClassification.from_pretrained('NER_model',
                                                   
                                                   num_labels=3)  ## 3
model = model.to(device)

In [42]:
# 设置模型为评估模式（不启用训练模式）
model.eval()

# 示例输入句子
input_text = "浦东申城佳苑1期C块旁边有一堆垃圾，希望能尽快清理。"

# 使用分词器对输入进行编码
inputs = tokenizer(input_text, return_tensors="pt")

inputs = inputs.to(device)

# 运行模型进行预测
with torch.no_grad():
    outputs = model(**inputs)

# 获取预测结果
predictions = torch.argmax(outputs.logits, dim=2).tolist()
prediction = predictions[0]
indices = [index for index, value in enumerate(prediction[1: -1]) if value == 1 or value == 2]
entity_loc = ''.join([input_text[idx] for idx in indices])


'浦东申城佳苑1期C块'